# 2. COORDINATION

In [1]:
import sys

sys.version

'3.14.0 free-threading build (main, Nov  1 2025, 00:37:47) [GCC 14.2.0]'

You are now in **Phase 2: Coordination**, where we focus on the sophisticated tools for synchronizing and communicating between threads.

Last session, you successfully used a `threading.Lock` to fix a data race. You saw that it makes a block of code atomic, preventing other threads from entering until the lock is released. This is the fundamental guarantee of a mutex.

Now, let's take the next small step and explore the performance implications of this guarantee.

# <b>2.1 Cost of Lock</b>

A Lock is a powerful tool, but it comes with a cost. The act of acquiring and releasing a lock requires work from the operating system and the Python runtime. More importantly, if a thread tries to acquire a lock that is already held, it is put to sleep — it **blocks**. This context switch (saving its state, scheduling another thread, later restoring it) is **one of the most expensive operations in a system**.

Consider this scenario: You have a shared counter and ten threads. Each thread needs to increment the counter 1,000,000 times.

**Question:** In a perfect world without any locking overhead, how many total operations would be performed? And what is the primary performance cost we introduce when we use a single lock to protect this counter?

**10,000,000 total operations.** And a **coarse-grained lock** (a single lock protecting a large or frequently accessed resource) can serialize execution, destroying parallelism and making threads wait unnecessarily.

Let's make this concrete. Here is a comparison table of the two locking strategies we can use for this counter problem:

| Characteristic | Coarse-Grained Lock (Single Lock) | Fine-Grained Locking (No Lock for Counter) |
| :--- | :--- | :--- |
| **Concept** | One lock protects the entire shared resource or a large section of code. | The problem is designed to avoid shared state, or locks protect very small, independent resources. |
| **Parallelism** | **Low.** Threads spend most of their time waiting. Effectively serial. | **High.** Threads do not block each other; true concurrent execution. |
| **Correctness** | Easy to reason about; data race is impossible. | Can be difficult; requires careful design to be correct. |
| **Performance** | **Poor.** High overhead from contention and context switching. | **Excellent.** Minimal overhead, linear speedup with more cores. |
| **Analogy** | A single, slow checkout line for an entire grocery store. | Every customer uses a self-checkout machine simultaneously. |

The "fine-grained" solution in this specific case is to avoid shared state altogether. Each thread can increment its own **local counter** and then, only once at the very end, acquire a lock to add its local total to the shared global counter. This reduces 10,000,000 lock acquisitions down to just 10.

# <b>2.2 RLock</b>

## Deadlock Example

Now, building on this understanding of lock granularity, let's introduce a new primitive: the **`threading.RLock`** (Re-entrant Lock).

An `RLock` allows the *same thread* to acquire the lock multiple times without deadlocking itself. It must be released the same number of times. This is useful when you have functions that call other functions which both require the same lock.

**Question:** Can you think of a scenario, perhaps involving a function that calls another function, where using a regular `Lock` would cause a thread to permanently block itself?

Imagine you have a shared data structure, like a list of transactions, protected by a lock. You have a function that needs to traverse this list, so it acquires the lock.

Now, what if that function, while holding the lock, needs to call a helper function — or even itself **recursively** — that *also* needs to be thread-safe and operate on the same list? The helper function will try to acquire the same lock that the main function is already holding.

Here is the code that demonstrates the problem:

In [8]:
import threading


def main():
    """Run code."""
    # Using a standard Lock
    lock = threading.Lock()

    def recursive_traverse(depth):
        print(f"Calling function: Depth {depth}")
        with lock:  # First acquisition here...
            if depth > 0:
                # ...Problem: We try to acquire the same lock AGAIN here!
                recursive_traverse(depth - 1)

    # This will cause a deadlock.
    # The thread will block forever on the inner `with lock` call.
    recursive_traverse(2)


if __name__ == "__main__":
    main()

Calling function: Depth 2
Calling function: Depth 1


KeyboardInterrupt: 

In [9]:
!python3.14t src/recursive_traverse_selflock.py

Calling function: Depth 2
Calling function: Depth 1
^C
Traceback (most recent call last):
  File "/home/commi/Yandex.Disk/it_working/projects/ai/python_threading_for_ai_course/02_coordination/src/recursive_traverse_selflock.py", line 22, in <module>
    main()
    ~~~~^^
  File "/home/commi/Yandex.Disk/it_working/projects/ai/python_threading_for_ai_course/02_coordination/src/recursive_traverse_selflock.py", line 18, in main
    recursive_traverse(2)
    ~~~~~~~~~~~~~~~~~~^^^
  File "/home/commi/Yandex.Disk/it_working/projects/ai/python_threading_for_ai_course/02_coordination/src/recursive_traverse_selflock.py", line 14, in recursive_traverse
    recursive_traverse(depth - 1)
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "/home/commi/Yandex.Disk/it_working/projects/ai/python_threading_for_ai_course/02_coordination/src/recursive_traverse_selflock.py", line 11, in recursive_traverse
    with lock:  # First acquisition here...
         ^^^^
KeyboardInterrupt


If you run this with a standard `Lock`, the thread will acquire the lock at the first call to `recursive_traverse(2)`. When it calls `recursive_traverse(1)`, the inner function will try to acquire the same lock. Since the thread already holds it, it will wait forever for itself to release it... which it never will. This is a **self-deadlock**.

## RLock example

**Question:** Based on this problem, what specific behavior do you think an `RLock` (Re-entrant Lock) must have that a regular `Lock` does not?

An `RLock` does not release the lock. Instead, it operates like a **counter** or a **recursion depth tracker** owned by the thread.

Here is the exact mechanism:

1.  **Acquisition:** When a thread acquires an `RLock` for the first time, it takes ownership and the internal counter is set to 1.
2.  **Re-acquisition:** If the *same thread* acquires the `RLock` again, the counter is simply incremented to 2. The thread does not block.
3.  **Release:** Each time the thread releases the `RLock`, the counter is decremented.
4.  **True Release:** Only when the counter reaches zero does the `RLock` actually become unlocked, allowing other threads to acquire it.

It does not create independent locks; it's still a single lock, but it remembers which thread owns it and how many times that thread has acquired it.

Let's fix the previous code:

In [11]:
import threading


def main():
    """Run code."""
    # Using an RLock
    r_lock = threading.RLock()

    def recursive_traverse(depth):
        print(f"Calling function: Depth {depth}")
        # Acquisition #1 (or #2, #3...) for this thread.
        with r_lock:
            if depth > 0:
                # Inner call acquires the same RLock successfully.
                recursive_traverse(depth - 1)
                # The RLock is only truly released after the outermost 'with' block exits.

    # This now works correctly and does not deadlock.
    recursive_traverse(2)


if __name__ == "__main__":
    main()

Calling function: Depth 2
Calling function: Depth 1
Calling function: Depth 0


The `RLock` "saves the information" (the acquisition count) so it knows how many times it must be released before it's free for other threads.

# <b>2.3 Debugging Deadlocks</b>

Moving from theory to practice, debugging concurrency issues is where the real challenge lies. Deadlocks, especially, are notorious because the program doesn't crash; it just... stops, silently.

Let's build a toolkit, starting with the simplest methods and moving to the more advanced. I'll use a classic example of a deadlock between two threads to demonstrate.

First, consider this code with an intentional deadlock:

```python
import threading
import time

lock_a = threading.Lock()
lock_b = threading.Lock()

def thread_1():
    with lock_a:
        print("Thread 1 acquired lock A")
        time.sleep(0.1)  # Critical: This pause ensures thread_2 can acquire lock B
        print("Thread 1 waiting to acquire lock B...")
        with lock_b:
            print("Thread 1 acquired both locks (A and B)")

def thread_2():
    with lock_b:
        print("Thread 2 acquired lock B")
        print("Thread 2 waiting to acquire lock A...")
        with lock_a:
            print("Thread 2 acquired both locks (A and B)")

t1 = threading.Thread(target=thread_1)
t2 = threading.Thread(target=thread_2)

t1.start()
t2.start()
t1.join()
t2.join()
print("This line will never be printed.")
```

**Question:** Before we get to the tools, based on your understanding, can you trace the sequence of operations that leads this program to freeze permanently?

This is the **Circular Wait** condition, one of the four Coffman conditions necessary for a deadlock.

*   **Thread 1** holds Lock A and requests Lock B.
*   **Thread 2** holds Lock B and requests Lock A.

Neither can proceed, so the program hangs. Now, let's look at the debugging tools, from simple to sophisticated.

## The "Poor Man's Debugger" (`print` & Logging)

The simplest method is to add detailed logging. The `threading` module can help.

The log output will show you the exact sequence leading to the deadlock and then stop, pointing you to the last operation before the hang.

In [1]:
import logging
import threading
import time


def main():
    """Run code."""
    logging.basicConfig(
        level=logging.DEBUG, format="%(relativeCreated)6d %(threadName)s %(message)s"
    )

    lock_a = threading.Lock()
    lock_b = threading.Lock()

    t1 = threading.Thread(target=thread, args=(lock_a, lock_b))
    t2 = threading.Thread(target=thread, args=(lock_b, lock_a))

    t1.start()
    t2.start()


def thread(lock_a, lock_b):
    """Start thread."""
    logging.debug("Started")
    with lock_a:
        logging.debug(f"Acquired lock {lock_a}")
        time.sleep(0.1)
        logging.debug(f"Trying to acquire lock {lock_b}...")
        with lock_b:
            logging.debug(f"Acuired both {lock_a} and {lock_b}")


if __name__ == "__main__":
    main()

  2914 Thread-5 (thread) Started
  2916 Thread-6 (thread) Started
  2918 Thread-5 (thread) Acquired lock <locked _thread.lock object at 0x5b604c41010>
  2920 Thread-6 (thread) Acquired lock <locked _thread.lock object at 0x5b604c40fd0>
  3021 Thread-5 (thread) Trying to acquire lock <locked _thread.lock object at 0x5b604c40fd0>...
  3026 Thread-6 (thread) Trying to acquire lock <locked _thread.lock object at 0x5b604c41010>...


In [2]:
!python3.14t src/debug_logging.py

     5 Thread-1 (thread) Started
     5 Thread-1 (thread) Acquired lock <locked _thread.lock object at 0x57b90603090>
     7 Thread-2 (thread) Started
     7 Thread-2 (thread) Acquired lock <locked _thread.lock object at 0x57b90600110>
   106 Thread-1 (thread) Trying to acquire lock <locked _thread.lock object at 0x57b90600110>...
   107 Thread-2 (thread) Trying to acquire lock <locked _thread.lock object at 0x57b90603090>...
^C
Exception ignored while joining a thread in _thread._shutdown():
Traceback (most recent call last):
  File "/usr/local/lib/python3.14t/threading.py", line 1582, in _shutdown
    _thread_shutdown()
KeyboardInterrupt: 


## Signaling with Timeouts

This is a more active debugging and recovery technique. You can attempt to acquire a lock with a timeout. If it fails, you log an error and potentially break the deadlock (though breaking it correctly is complex).

In [3]:
import logging
import threading
import time


def main():
    """Run code."""
    logging.basicConfig(
        level=logging.DEBUG, format="%(relativeCreated)6d %(threadName)s %(message)s"
    )

    lock_a = threading.Lock()
    lock_b = threading.Lock()

    t1 = threading.Thread(target=thread, args=(lock_a, lock_b))
    t2 = threading.Thread(target=thread, args=(lock_b, lock_a))

    t1.start()
    t2.start()


def thread(lock_a, lock_b):
    """Start thread."""
    logging.debug("Started")
    with lock_a:
        logging.debug(f"Acquired lock {lock_a}")
        time.sleep(0.1)
        logging.debug(f"Trying to acquire lock {lock_b}...")
        # Try to acquire lock B, but only wait for 2 seconds.
        acquired = lock_b.acquire(timeout=2)
        if acquired:
            logging.debug(f"Acuired both {lock_a} and {lock_b}")
        else:
            logging.error(f"COULD NOT ACQUIRE LOCK {lock_b}! POTENTIAL DEADLOCK!")
        # Remember to release lock_b if the logic requires it.
    logging.debug("Thread exiting")


if __name__ == "__main__":
    main()

451678 Thread-7 (thread) Started
451679 Thread-8 (thread) Started
451682 Thread-7 (thread) Acquired lock <locked _thread.lock object at 0x5b604c42590>
451686 Thread-8 (thread) Acquired lock <locked _thread.lock object at 0x5b604c425d0>
451788 Thread-7 (thread) Trying to acquire lock <locked _thread.lock object at 0x5b604c425d0>...
451791 Thread-8 (thread) Trying to acquire lock <locked _thread.lock object at 0x5b604c42590>...
453789 Thread-7 (thread) COULD NOT ACQUIRE LOCK <locked _thread.lock object at 0x5b604c425d0>! POTENTIAL DEADLOCK!
453791 Thread-7 (thread) Thread exiting
453791 Thread-8 (thread) Acuired both <locked _thread.lock object at 0x5b604c425d0> and <locked _thread.lock object at 0x5b604c42590>
453792 Thread-8 (thread) Thread exiting


In [6]:
!python3.14t src/debug_timeouts.py

     4 Thread-1 (thread) Started
     5 Thread-1 (thread) Acquired lock <locked _thread.lock object at 0x469b6603090>
    13 Thread-2 (thread) Started
    13 Thread-2 (thread) Acquired lock <locked _thread.lock object at 0x469b6600110>
   105 Thread-1 (thread) Trying to acquire lock <locked _thread.lock object at 0x469b6600110>...
   117 Thread-2 (thread) Trying to acquire lock <locked _thread.lock object at 0x469b6603090>...
  2108 Thread-1 (thread) COULD NOT ACQUIRE LOCK <locked _thread.lock object at 0x469b6600110>! POTENTIAL DEADLOCK!
  2109 Thread-1 (thread) Thread exiting
  2109 Thread-2 (thread) Acuired both <locked _thread.lock object at 0x469b6600110> and <locked _thread.lock object at 0x469b6603090>
  2109 Thread-2 (thread) Thread exiting


In [7]:
!python3.14t src/debug_timeouts.py

     2 Thread-1 (thread) Started
     2 Thread-1 (thread) Acquired lock <locked _thread.lock object at 0x5b974603090>
     2 Thread-2 (thread) Started
     2 Thread-2 (thread) Acquired lock <locked _thread.lock object at 0x5b974600110>
   102 Thread-1 (thread) Trying to acquire lock <locked _thread.lock object at 0x5b974600110>...
   103 Thread-2 (thread) Trying to acquire lock <locked _thread.lock object at 0x5b974603090>...
  2103 Thread-1 (thread) COULD NOT ACQUIRE LOCK <locked _thread.lock object at 0x5b974600110>! POTENTIAL DEADLOCK!
  2103 Thread-2 (thread) COULD NOT ACQUIRE LOCK <locked _thread.lock object at 0x5b974603090>! POTENTIAL DEADLOCK!
  2103 Thread-1 (thread) Thread exiting
  2104 Thread-2 (thread) Thread exiting


## The Professional's Tool: `gdb`

For a program that's already hung, you need to see what each thread is *doing*. The best way is to get a **thread dump**.

For ultimate control, you can use a debugger like `gdb` to attach to the running process and get a full backtrace.

Here is the step-by-step protocol:

1.  **First, run your deadlocking Python script in one terminal window and let it hang.**
    ```bash
    python3.14t debug_logging.py
    ```
    (It will print the first few lines and then hang, just as you showed).

2.  **Open a second terminal window.** Find the Process ID (PID) of your running Python script.
    ```bash
    ps aux | grep debug_logging.py
    ```
    You'll get output like:
    ```
    user     12345  0.5  0.1 287312 14532 pts/0    Sl+  14:30   0:00 python3.14t debug_logging.py
    ```
    The number `12345` is the PID.

3.  **Attach the GNU Debugger (`gdb`) to the running process.**
    ```bash
    sudo gdb -p 12345
    ```
    (You need `sudo` because `gdb` attaches to another user's process, even if it's your own).

4.  **Inside `gdb`, run this command to get a full backtrace of all threads:**
    ```
    (gdb) thread apply all bt
    ```
    This is the magic command. It applies the `bt` (backtrace) command to *all* threads.

5.  **Analyze the output.** You will see a dump for each thread. The ones stuck in `pthread_cond_wait` or `sem_wait` are your waiting threads. Crucially, you should also see frames from the Python interpreter, like `PyThread_acquire_lock_timed`, which will tell you exactly which Python lock a thread is blocked on.

6.  **Detach and quit `gdb` without killing your program:**
    ```
    (gdb) detach
    (gdb) quit
    ```

This `gdb` method is the gold standard for diagnosing a stuck production process. It doesn't require any code changes and gives you a low-level, undeniable picture of what every thread is doing.

### `gdb` debugging example

```
(gdb) thread apply all bt

Thread 3 (Thread 0x7f8bb45ff6c0 (LWP 173374) "Thread-1 (threa"):
#11 0x0000559500ab2de6 in _PyMutex_LockTimed (m=m@entry=0x548b4600130, timeout=<optimized out>, flags=flags@entry=(_PY_LOCK_DETACH | _PY_LOCK_HANDLE_SIGNALS)) at Python/lock.c:108
#12 0x0000559500b81354 in lock_PyThread_acquire_lock (op=0x548b4600110, args=<optimized out>, kwds=<optimized out>) at ./Modules/_threadmodule.c:823

Thread 2 (Thread 0x7f8bb3dfe6c0 (LWP 173375) "Thread-2 (threa"):
#11 0x0000559500ab2de6 in _PyMutex_LockTimed (m=m@entry=0x548b46030b0, timeout=<optimized out>, flags=flags@entry=(_PY_LOCK_DETACH | _PY_LOCK_HANDLE_SIGNALS)) at Python/lock.c:108
--Type <RET> for more, q to quit, c to continue without paging--
#12 0x0000559500b81354 in lock_PyThread_acquire_lock (op=0x548b4603090, args=<optimized out>, kwds=<optimized out>) at ./Modules/_threadmodule.c:823


Thread 1 (Thread 0x7f8bb49a7780 (LWP 173373) "python3.14t"):
#11 0x0000559500b4f42c in PyEvent_WaitTimed (evt=evt@entry=0x5595143fd5a9, timeout_ns=timeout_ns@entry=-1, detach=detach@entry=1) at Python/lock.c:294
#12 0x0000559500b81edd in ThreadHandle_join (self=self@entry=0x5595143fd570, timeout_ns=timeout_ns@entry=-1) at ./Modules/_threadmodule.c:540

(gdb) detach
Detaching from program: /usr/local/bin/python3.14t, process 173373
[Inferior 1 (process 173373) detached]
(gdb) quit
```

This log is a goldmine. The key is to ignore the complex low-level `futex` and `sem_wait` calls and look for the **Python-specific function names** and the **memory addresses** of the locks.

Let's analyze your `gdb` output:

**Thread 3 (Thread-1) is stuck here:**
`#12 0x0000559500b81354 in lock_PyThread_acquire_lock (op=0x548b4600110, args=<optimized out>, kwds=<optimized out>) at ./Modules/_threadmodule.c:823`

*   `lock_PyThread_acquire_lock` is the C function that implements `lock.acquire()`.
*   The argument `op=0x548b4600110` is the memory address of the Python lock object it's trying to acquire.

**Thread 2 (Thread-2) is stuck here:**
`#12 0x0000559500b81354 in lock_PyThread_acquire_lock (op=0x548b4603090, args=<optimized out>, kwds=<optimized out>) at ./Modules/_threadmodule.c:823`

*   Same function, different lock object: `op=0x548b4603090`.

**Now, the critical link:** Look at the frame *above* the acquire call for each thread to see *which* lock they are holding. It's in the `_PyMutex_LockTimed` call.

*   **Thread 3** (from `#11`): `_PyMutex_LockTimed (m=m@entry=0x548b4600130 ...)`
*   **Thread 2** (from `#11`): `_PyMutex_LockTimed (m=m@entry=0x548b46030b0 ...)`

**The Proof:**
Let's map these addresses back to your Python code:

*   `lock_a` in Python is represented by the C object at `0x548b4600110` and its internal mutex at `0x548b4600130`.
*   `lock_b` in Python is represented by the C object at `0x548b4603090` and its internal mutex at `0x548b46030b0`.

**The Deadlock, Proven:**

1.  **Thread 3 (Thread-1)** holds the mutex for `lock_a` (`0x548b4600130`) and is blocked in `acquire` waiting for `lock_b` (`0x548b4603090`).
2.  **Thread 2 (Thread-2)** holds the mutex for `lock_b` (`0x548b46030b0`) and is blocked in `acquire` waiting for `lock_a` (`0x548b4600130`).

**Thread 1** is the main thread, which is simply blocked waiting to join the other two threads (`ThreadHandle_join`).

This is irrefutable, low-level evidence of the circular wait deadlock we theorized. You have now used a systems-level tool to prove a high-level logic error.

This skill — extracting the relevant story from a verbose systems log — is a superpower.

## Summary Table

| Method | Use Case | Pros | Cons |
| :--- | :--- | :--- | :--- |
| **Logging (`print`)** | Proactive, during development. | Simple, no extra tools. | Can change timing ("Heisenbug"), verbose. |
| **Timeouts** | Proactive debugging & resilient systems. | Can detect and potentially recover. | Adds complexity, recovery logic is hard. |
| **`faulthandler`/`SIGQUIT`** | **Reactive**, for a already-hung program. | Shows the exact state of a frozen program. | Requires terminal/diagnostics access. |
| **Debugger (`gdb`)** | Deep, reactive inspection. | Most powerful, can inspect memory. | Complex, requires system-level knowledge. |

The best strategy is to use logging proactively to understand your thread flow, and keep `faulthandler` in your back pocket for when things inevitably go wrong.

# <b>Conclusion</b>

You have now mastered a critical part of **Phase 2: Coordination**. Let's solidify what you've accomplished in this session:

*   **Deepened Lock Understanding:** You analyzed the performance trade-off between coarse-grained and fine-grained locking.
*   **Mastered RLock:** You understood the problem of re-entrant code and how `RLock` solves it with an internal counter.
*   **Built a Debugging Toolkit:** You progressed from simple logging to using `gdb` to get a definitive thread dump from a frozen process, and you learned to trace the deadlock to specific lock addresses.

# <b>Reading</b>

1.  **Article:** "Debugging Deadlocks in Python" by Real Python. (A great practical companion to what we did today).
2.  **Video:** "Debugging with GDB - StackTraces and Threads" by The Linux Foundation. (To build on your new `gdb` skills).
3.  **Book Chapter:** "The Linux Programming Interface", Chapters 31-33, by Michael Kerrisk. (The definitive guide to pthreads and synchronization, which directly maps to Python's underlying mechanics).